# <font color= F30720> <b> <i> Modelando la serie de la temperatura </i> </b> </font>
La serie de la temperatura fue extraída de la base de datos del IDEAM, se tuvieron en cuenta los promedios diarios de las temperaturas en grados centigrados (°C) registradas en Bogotá en las diferentes estaciones meteorológicas que recolectaron información de esos días.
La serie de tiempo cuenta con un total de 1826 registros, de los cuales 13 (0.7%) fueron imputados puesto que no se presentaba la información necesaria. Esta imputación fue realizada a partir del método de vecinos más cercanos (KNN), donde se tuvierón en cuenta 5 vecinos.

Para el modelar la serie de la temperatura se usara inicialmente un modelo SARIMA, seguido de redes recurrentes simples, SRNN y finalmente GRU.
Para seleccionar el mejor modelo se tomará como criterio el error cuadrático medio.

## <font color= 199EDC> <b> Red Neuronal Recurrente Simple </b> </font>

#### <font color= blue>  Importación de Datos</font>

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

import sklearn as sk
from sklearn import impute
from sklearn import preprocessing
import sklearn.externals
import joblib
from sklearn.model_selection import TimeSeriesSplit
from sklearn.impute import KNNImputer
import sklearn.preprocessing

from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import time
import sklearn.externals
import joblib
import plotly.graph_objects as go
from sklearn import metrics

import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from statsmodels.tsa.statespace.sarimax import SARIMAX

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow.keras.backend as K

%matplotlib inline

In [ ]:
import plotly.io as pio
pio.renderers.default='notebook'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/Shareddrives/Mineria /Temperatura1.csv", sep=';', header=0, decimal = ',')
Fecha = pd.date_range(start='2017-01-01', end='2021-12-31', freq='D')
df['Fecha'] = Fecha
df = df.set_index('Fecha')

print(df[pd.isnull(df.ValorObservado)])
print('En total hay' ,
      str(df['ValorObservado'].isnull().sum()) ,
      'valores sin información')
print('Correspondientes al {:.3f}% del total'
      .format(df['ValorObservado'].isnull().sum()*100/len(df)))

            ValorObservado
Fecha                     
2017-08-12             NaN
2017-12-24             NaN
2019-09-15             NaN
2019-09-16             NaN
2019-09-17             NaN
2020-11-12             NaN
2021-01-05             NaN
2021-01-06             NaN
2021-01-07             NaN
2021-01-08             NaN
2021-08-18             NaN
2021-08-20             NaN
2021-12-05             NaN
En total hay 13 valores sin información
Correspondientes al 0.712% del total


La serie presenta valores faltantes, por lo tantol se imputaran usando el método de vecinos más cercanos (KNN), como se muestra a continuación.

#### <font color= blue> Imputación a partir del vecino más cercano</font>

In [4]:
#Imputación de Valores usando el vecino más cercano
imput = KNNImputer(n_neighbors=5, weights="uniform")

# Ajustamos el modelo e imputamos los missing values
imput.fit(df[['ValorObservado']])
df['ValorObservado'] = imput.transform(df[['ValorObservado']]).ravel()
print()
print("Valores pérdidos en ValorObservado: " , 
      str(df['ValorObservado'].isnull().sum()))


Valores pérdidos en ValorObservado:  0


In [5]:
fig = px.line(df, x=df.index, y="ValorObservado")
fig.update_xaxes(title_text="Fecha")
fig.show()

#### <font color= blue> Separación de datos entrenamiento y validación  </font>
Para el respectivo análisis se tomarán el 80% de los datos para entrenamiento y validacion, el 20% restantes para prueba, dichos valores corresponden a 1460 y 365 respectivamente.

In [6]:
from sklearn.preprocessing import MinMaxScaler
# crea el objeto  scaler y escala los datos
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df.values)
#
df_norm = pd.DataFrame(scaled_data,index=df.index, columns=['ValorObservadoNormalizado'])

In [7]:
train_size = int(len(df_norm) * 0.8)
test_size = len(df_norm) - train_size
train, test = df_norm.iloc[0:train_size], df_norm.iloc[train_size:len(df_norm)]
len_train = len(train)
len_test = len(test)
print(len_train, len_test)

1460 366


In [8]:
def create_dataset(X, y, time_steps=1):
    # crea dos listas vacias para depositar los datos
    Xs, ys = [], []
    # el primer lote de datos empieza en la primera observación
    # y toma time_steps  datos.
    # Comienza a avanzar hacia adelante.
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [9]:
time_steps = 50

# reshape to [samples, time_steps, n_features]

X_train, y_train = create_dataset(train, train, time_steps)
X_test, y_test = create_dataset(test, test, time_steps)

print("X_train.shape = ", X_train.shape)
print("y_train.shape = ", y_train.shape)
print("X_test.shape = ", X_test.shape)
print("y_test.shape = ", y_test.shape)

X_train.shape =  (1410, 50, 1)
y_train.shape =  (1410, 1)
X_test.shape =  (316, 50, 1)
y_test.shape =  (316, 1)


In [10]:
fig = px.line(df_norm, x=df.index, y='ValorObservadoNormalizado')
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="ValorObservadoNormalizado")
fig.show()

#### <font color= blue> Modelo (1 paso adelante) </font>

In [11]:
from sklearn.utils import shuffle
# shapes
inputs_shape = (X_train.shape[1], X_train.shape[2])
SRNN_output = 60

# layers
inputs = L.Input(inputs_shape)
srnn = L.SimpleRNN(units=SRNN_output, name='SRNN')(inputs)
outputs_SRNN = L.Dense(1)(srnn)

SRNN_model = Model(inputs=inputs, outputs=outputs_SRNN, name='series_SRNN_model')

# Compiling the RNN
SRNN_model.compile(loss="mean_squared_error", optimizer=Adam(0.001))
# Fitting to the training set
start = time.time()
SRNN = SRNN_model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=False
)
print("compilation time : ", time.time() - start)

Epoch 1/50
80/80 [==============================] - 4s 22ms/step - loss: 0.0286 - val_loss: 0.0298
Epoch 2/50
80/80 [==============================] - 1s 13ms/step - loss: 0.0168 - val_loss: 0.0103
Epoch 3/50
80/80 [==============================] - 1s 10ms/step - loss: 0.0114 - val_loss: 0.0036
Epoch 4/50
80/80 [==============================] - 1s 10ms/step - loss: 0.0071 - val_loss: 0.0035
Epoch 5/50
80/80 [==============================] - 1s 11ms/step - loss: 0.0067 - val_loss: 0.0035
Epoch 6/50
80/80 [==============================] - 1s 12ms/step - loss: 0.0065 - val_loss: 0.0035
Epoch 7/50
80/80 [==============================] - 2s 21ms/step - loss: 0.0063 - val_loss: 0.0034
Epoch 8/50
80/80 [==============================] - 1s 17ms/step - loss: 0.0062 - val_loss: 0.0034
Epoch 9/50
80/80 [==============================] - 1s 11ms/step - loss: 0.0061 - val_loss: 0.0033
Epoch 10/50
80/80 [==============================] - 1s 10ms/step - loss: 0.0061 - val_loss: 0.0033
Epoch 11/

In [12]:
joblib.dump(SRNN_model,'/content/drive/Shareddrives/Mineria /SRNN')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......input_layer
.........vars
......simple_rnn
.........cell
............vars
...............0
...............1
...............2
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
variables.h5                                   2023-02-20 02:20:46        68504
config.json                                    2023-02-20 02:20:46         2436
metadata.json                                  2023-02-20 02:20:46           64


['/content/drive/Shareddrives/Mineria /SRNN']

In [13]:
SRNN_model.summary()

Model: "series_SRNN_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50, 1)]           0         
                                                                 
 SRNN (SimpleRNN)            (None, 60)                3720      
                                                                 
 dense (Dense)               (None, 1)                 61        
                                                                 
Total params: 3,781
Trainable params: 3,781
Non-trainable params: 0
_________________________________________________________________


In [14]:
SRNN_losses = pd.DataFrame(SRNN.history)
fig = px.line(SRNN_losses, x=SRNN_losses.index, y=["loss", "val_loss"])
fig.update_xaxes(title_text="Epoch")
fig.update_yaxes(title_text="Loss")
fig.show()

#### <font color= blue> Predicciones (1 paso adelante)</font>

In [15]:
SRNN_Predict = SRNN_model.predict(X_test)
SRNN_Predict = scaler.inverse_transform(SRNN_Predict)

10/10 [==============================] - 0s 4ms/step


In [16]:
# plot y_train, y_test, and testPredict using plotly
seq_len=50
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df.index[seq_len : len(y_train) + seq_len],
        y=scaler.inverse_transform(y_train).ravel(),
        mode="lines",
        name="Entrenamiento",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train) + seq_len :],
        y=scaler.inverse_transform(y_test).ravel(),
        mode="lines",
        name="Prueba",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train) + seq_len :],
        y=SRNN_Predict.ravel(),
        mode="lines",
        name="Predicción",
    )
)
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="ValorObservado")
fig.show()

#### <font color= blue>  Intervalos de Confianza (1 paso adelante) </font>

In [18]:
def QuantileLoss(perc, delta=1e-4):
    perc = np.array(perc).reshape(-1)
    perc.sort()
    perc = perc.reshape(1, -1)
    def _qloss(y, pred):
        I = tf.cast(y <= pred, tf.float32)
        d = K.abs(y - pred)
        correction = I * (1 - perc) + (1 - I) * perc
        # huber loss
        huber_loss = K.sum(correction * tf.where(d <= delta, 0.5 * d ** 2 / delta, d - 0.5 * delta), -1)
        # order loss
        q_order_loss = K.sum(K.maximum(0.0, pred[:, :-1] - pred[:, 1:] + 1e-6), -1)
        return huber_loss + q_order_loss
    return _qloss

In [19]:
# quantiles
perc_points = [0.025, 0.975]

# shapes
inputs_shape = (X_train.shape[1], X_train.shape[2])
SRNN_output = 60

# layers
inputs = L.Input(inputs_shape)
qsrnn = L.SimpleRNN(units=SRNN_output, name='SRNN')(inputs)
qoutputs_SRNN = L.Dense(2)(qsrnn)

qSRNN_model = Model(inputs=inputs, outputs=qoutputs_SRNN, name='series_SRNN_model')

# Compiling the RNN
qSRNN_model.compile(Adam(0.001), loss=QuantileLoss(perc_points))
# Fitting to the training set
start = time.time()
qSRNN = qSRNN_model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=False
)
print("compilation time : ", time.time() - start)

Epoch 1/50
80/80 [==============================] - 4s 23ms/step - loss: 0.0279 - val_loss: 0.0115
Epoch 2/50
80/80 [==============================] - 1s 10ms/step - loss: 0.0163 - val_loss: 0.0104
Epoch 3/50
80/80 [==============================] - 1s 11ms/step - loss: 0.0196 - val_loss: 0.0126
Epoch 4/50
80/80 [==============================] - 1s 11ms/step - loss: 0.0134 - val_loss: 0.0091
Epoch 5/50
80/80 [==============================] - 1s 11ms/step - loss: 0.0144 - val_loss: 0.0095
Epoch 6/50
80/80 [==============================] - 1s 11ms/step - loss: 0.0117 - val_loss: 0.0087
Epoch 7/50
80/80 [==============================] - 1s 11ms/step - loss: 0.0113 - val_loss: 0.0088
Epoch 8/50
80/80 [==============================] - 1s 11ms/step - loss: 0.0119 - val_loss: 0.0104
Epoch 9/50
80/80 [==============================] - 1s 11ms/step - loss: 0.0109 - val_loss: 0.0094
Epoch 10/50
80/80 [==============================] - 1s 11ms/step - loss: 0.0117 - val_loss: 0.0085
Epoch 11/

In [20]:
qSRNN_model.summary()

Model: "series_SRNN_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 50, 1)]           0         
                                                                 
 SRNN (SimpleRNN)            (None, 60)                3720      
                                                                 
 dense_1 (Dense)             (None, 2)                 122       
                                                                 
Total params: 3,842
Trainable params: 3,842
Non-trainable params: 0
_________________________________________________________________


In [21]:
qSRNN_losses = pd.DataFrame(qSRNN.history)
fig = px.line(qSRNN_losses, x=qSRNN_losses.index, y=["loss", "val_loss"])
fig.update_xaxes(title_text="Epoch")
fig.update_yaxes(title_text="Loss")
fig.show()

In [22]:
qSRNN_Predict = qSRNN_model.predict(X_test)
qSRNN_Predict = scaler.inverse_transform(qSRNN_Predict)

10/10 [==============================] - 0s 7ms/step


In [23]:
# plot y_train, y_test, and testPredict using plotly
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df.index[seq_len : len(y_train) + seq_len],
        y=scaler.inverse_transform(y_train).ravel(),
        mode="lines",
        name="Entrenamiento",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train) + seq_len :],
        y=SRNN_Predict.ravel(),
        mode="lines",
        name="Predicción",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train) + seq_len :],
        y=qSRNN_Predict[:,0] ,
        mode="lines",
        name="0.025",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train) + seq_len :],
        y=qSRNN_Predict[:,1] ,
        mode="lines",
        name="0.975",
    )
)
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="ValorObservado")
fig.show()

#### <font color= blue>  Error Cuadrático Medio (1 paso adelante) </font>

In [24]:
SRNN_Score1 = metrics.mean_squared_error(scaler.inverse_transform(y_test), SRNN_Predict)
print('Test Score: %.2f RMSE' % (SRNN_Score1))

Test Score: 0.74 RMSE


#### <font color= blue>  Predicción 5 pasos adelante  </font>

In [25]:
def create_dataset(X, y, time_steps=1):
    # crea dos listas vacias para depositar los datos
    Xs, ys = [], []
    # el primer lote de datos empieza en la primera observación
    # y toma time_steps  datos.
    # Comienza a avanzar hacia adelante.
    for i in range(len(X) - time_steps-5):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps+5])
    return np.array(Xs), np.array(ys)

In [26]:
time_steps = 50

# reshape to [samples, time_steps, n_features]

X_train_5p, y_train_5p = create_dataset(train, train, time_steps)
X_test_5p, y_test_5p = create_dataset(test, test, time_steps)

print("X_train.shape = ", X_train_5p.shape)
print("y_train.shape = ", y_train_5p.shape)
print("X_test.shape = ", X_test_5p.shape)
print("y_test.shape = ", y_test_5p.shape)

X_train.shape =  (1405, 50, 1)
y_train.shape =  (1405, 1)
X_test.shape =  (311, 50, 1)
y_test.shape =  (311, 1)


In [27]:
# shapes
inputs_shape = (X_train_5p.shape[1], X_train_5p.shape[2])
SRNN_output_5p = 60

# layers
inputs_5p = L.Input(inputs_shape)
srnn_5p = L.SimpleRNN(units=SRNN_output_5p, name='SRNN')(inputs_5p)
outputs_SRNN_5p = L.Dense(1)(srnn_5p)

SRNN_model_5p = Model(inputs=inputs_5p, outputs=outputs_SRNN_5p, name='series_SRNN_model')

# Compiling the RNN
SRNN_model_5p.compile(optimizer=Adam(0.001),loss="mean_squared_error")
# Fitting to the training set
start = time.time()
SRNN_5p = SRNN_model_5p.fit(
    X_train_5p,
    y_train_5p,
    epochs=50,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=False
)
print("compilation time : ", time.time() - start)

Epoch 1/50
79/79 [==============================] - 2s 12ms/step - loss: 0.0389 - val_loss: 0.0175
Epoch 2/50
79/79 [==============================] - 1s 9ms/step - loss: 0.0198 - val_loss: 0.0142
Epoch 3/50
79/79 [==============================] - 1s 9ms/step - loss: 0.0171 - val_loss: 0.0071
Epoch 4/50
79/79 [==============================] - 1s 9ms/step - loss: 0.0140 - val_loss: 0.0060
Epoch 5/50
79/79 [==============================] - 1s 9ms/step - loss: 0.0135 - val_loss: 0.0057
Epoch 6/50
79/79 [==============================] - 1s 9ms/step - loss: 0.0132 - val_loss: 0.0055
Epoch 7/50
79/79 [==============================] - 1s 9ms/step - loss: 0.0129 - val_loss: 0.0054
Epoch 8/50
79/79 [==============================] - 1s 9ms/step - loss: 0.0127 - val_loss: 0.0053
Epoch 9/50
79/79 [==============================] - 1s 9ms/step - loss: 0.0126 - val_loss: 0.0053
Epoch 10/50
79/79 [==============================] - 1s 9ms/step - loss: 0.0124 - val_loss: 0.0052
Epoch 11/50
79/79 

In [ ]:
joblib.dump(SRNN_model_5p,'/content/drive/Shareddrives/Mineria /SRNN_r')

['/content/drive/Shareddrives/Mineria /SRNN_r']

In [28]:
SRNN_model_5p.summary()

Model: "series_SRNN_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 50, 1)]           0         
                                                                 
 SRNN (SimpleRNN)            (None, 60)                3720      
                                                                 
 dense_2 (Dense)             (None, 1)                 61        
                                                                 
Total params: 3,781
Trainable params: 3,781
Non-trainable params: 0
_________________________________________________________________


In [29]:
SRNN_losses_5p = pd.DataFrame(SRNN_5p.history)
fig = px.line(SRNN_losses_5p, x=SRNN_losses_5p.index, y=["loss", "val_loss"])
fig.update_xaxes(title_text="Epoch")
fig.update_yaxes(title_text="Loss")
fig.show()

In [30]:
SRNN_Predict_5p = SRNN_model.predict(X_test_5p )
SRNN_Predict_5p = scaler.inverse_transform(SRNN_Predict_5p)


10/10 [==============================] - 0s 4ms/step


In [31]:
# plot y_train, y_test, and testPredict using plotly
seq_len = 50
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df.index[seq_len : len(y_train_5p) + seq_len],
        y=scaler.inverse_transform(y_train_5p).ravel(),
        mode="lines",
        name="Entrenamiento",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train_5p) + seq_len :],
        y=scaler.inverse_transform(y_test_5p).ravel(),
        mode="lines",
        name="Prueba",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train_5p) + seq_len :],
        y=SRNN_Predict_5p.ravel(),
        mode="lines",
        name="Predicción",
    )
)
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="ValorObservado")
fig.show()

In [32]:
SRNN_Score2 = metrics.mean_squared_error(scaler.inverse_transform(y_test_5p), SRNN_Predict_5p)
print('Test Score: %.2f RMSE' % (SRNN_Score2))

Test Score: 1.57 RMSE


#### <font color= blue>  Tomando más retardos   </font>

In [33]:
def create_dataset(X, y, time_steps=1):
    # crea dos listas vacias para depositar los datos
    Xs, ys = [], []
    # el primer lote de datos empieza en la primera observación
    # y toma time_steps  datos.
    # Comienza a avanzar hacia adelante.
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [34]:
time_steps1 = 100

# reshape to [samples, time_steps, n_features]

X_train_r, y_train_r = create_dataset(train, train, time_steps1)
X_test_r, y_test_r = create_dataset(test, test, time_steps1)

print("X_train.shape = ", X_train_r.shape)
print("y_train.shape = ", y_train_r.shape)
print("X_test.shape = ", X_test_r.shape)
print("y_test.shape = ", y_test_r.shape)

X_train.shape =  (1360, 100, 1)
y_train.shape =  (1360, 1)
X_test.shape =  (266, 100, 1)
y_test.shape =  (266, 1)


In [35]:
# shapes
inputs_shape = (X_train_r.shape[1], X_train_r.shape[2])
SRNN_output_r = 60

# layers
inputs_r = L.Input(inputs_shape)
srnn_r = L.SimpleRNN(units=SRNN_output_r, name='SRNN')(inputs_r)
outputs_SRNN_r = L.Dense(1)(srnn_r)

SRNN_model_r = Model(inputs=inputs_r, outputs=outputs_SRNN_r, name='series_SRNN_model')

# Compiling the RNN
SRNN_model_r.compile(optimizer=Adam(0.001),loss="mean_squared_error")
# Fitting to the training set
start = time.time()
SRNN_r = SRNN_model_r.fit(
    X_train_r,
    y_train_r,
    epochs=50,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=False
)
print("compilation time : ", time.time() - start)

Epoch 1/50
77/77 [==============================] - 3s 20ms/step - loss: 0.0127 - val_loss: 0.0098
Epoch 2/50
77/77 [==============================] - 3s 33ms/step - loss: 0.0087 - val_loss: 0.0044
Epoch 3/50
77/77 [==============================] - 2s 31ms/step - loss: 0.0070 - val_loss: 0.0035
Epoch 4/50
77/77 [==============================] - 1s 17ms/step - loss: 0.0062 - val_loss: 0.0036
Epoch 5/50
77/77 [==============================] - 1s 18ms/step - loss: 0.0061 - val_loss: 0.0036
Epoch 6/50
77/77 [==============================] - 1s 18ms/step - loss: 0.0060 - val_loss: 0.0035
Epoch 7/50
77/77 [==============================] - 1s 17ms/step - loss: 0.0059 - val_loss: 0.0035
Epoch 8/50
77/77 [==============================] - 1s 18ms/step - loss: 0.0059 - val_loss: 0.0034
Epoch 9/50
77/77 [==============================] - 1s 18ms/step - loss: 0.0058 - val_loss: 0.0034
Epoch 10/50
77/77 [==============================] - 1s 18ms/step - loss: 0.0058 - val_loss: 0.0034
Epoch 11/

In [ ]:
joblib.dump(SRNN_model_5p,'/content/drive/Shareddrives/Mineria /SRNN_r')

['/content/drive/Shareddrives/Mineria /SRNN_r']

In [36]:
SRNN_model_r.summary()

Model: "series_SRNN_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 100, 1)]          0         
                                                                 
 SRNN (SimpleRNN)            (None, 60)                3720      
                                                                 
 dense_3 (Dense)             (None, 1)                 61        
                                                                 
Total params: 3,781
Trainable params: 3,781
Non-trainable params: 0
_________________________________________________________________


In [37]:
SRNN_losses_r = pd.DataFrame(SRNN_r.history)
fig = px.line(SRNN_losses_r, x=SRNN_losses_r.index, y=["loss", "val_loss"])
fig.update_xaxes(title_text="Epoch")
fig.update_yaxes(title_text="Loss")
fig.show()

In [38]:
SRNN_Predict_r = SRNN_model_r.predict(X_test_r)
SRNN_Predict_r = scaler.inverse_transform(SRNN_Predict_r)


9/9 [==============================] - 0s 8ms/step


In [39]:
# plot y_train, y_test, and testPredict using plotly
seq_len1=100
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df.index[seq_len1 : len(y_train_r) + seq_len1],
        y=scaler.inverse_transform(y_train_r).ravel(),
        mode="lines",
        name="Entrenamiento",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train_r) + seq_len1 :],
        y=scaler.inverse_transform(y_test_r).ravel(),
        mode="lines",
        name="Prueba",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train_r) + seq_len1 :],
        y=SRNN_Predict_r.ravel(),
        mode="lines",
        name="Predicción",
    )
)
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="ValorObservado")
fig.show()

In [40]:
SRNN_Score3 = metrics.mean_squared_error(scaler.inverse_transform(y_test_r), SRNN_Predict_r)
print('Test Score: %.2f RMSE' % (SRNN_Score3))

Test Score: 0.66 RMSE


#### <font color= blue>  Comparación   </font>

In [41]:
print('Modelo SRNN 1 paso adelante 50 retardos %f' % (SRNN_Score1))
print('Modelo SRNN 5 paso adelante 50 retardos %f' % (SRNN_Score2))
print('Modelo SRNN 1 paso adelante 100 retardos %f' % (SRNN_Score3))

Modelo SRNN 1 paso adelante 50 retardos 0.738110
Modelo SRNN 5 paso adelante 50 retardos 1.565561
Modelo SRNN 1 paso adelante 100 retardos 0.657670
